# Compare HSC and VISTA Z and Y aperture mags

In [1]:
from astropy.table import Table
import numpy as np


import matplotlib
import matplotlib.gridspec as gridspec 
import matplotlib.pyplot as plt

In [2]:
cat = Table.read('./data/HSC_z_y_comparison.fits')

FileNotFoundError: [Errno 2] No such file or directory: './data/HSC_z_y_comparison.fits'

In [ ]:
cat[:5]

In [ ]:
len(cat)

In [ ]:
np.sum(cat['flag'])

In [ ]:
plt.hist(cat['m_ap60_vista_z'][~np.isinf(cat['m_ap60_vista_z'])], bins=100)
plt.yscale('log')

In [ ]:
!mkdir figs

In [ ]:
def mag_to_flux(magnitudes, errors_on_magnitudes=None):
    """Convert AB magnitudes and errors to fluxes
    Given AB magnitudes and associated errors, this function returns the
    corresponding fluxes and associated flux errors (in Jy).
    The fluxes are computed with this formula:
        F = 10^((8.9 - Mag)/2.5)
    and the errors on fluxes with this one:
        F_err = ln(10)/2.5 * F * Mag_err
    Parameters
    ----------
    magnitudes: float or array-like of floats
        AB magnitudes of the sources.
    errors_on_magnitudes: float or array-like of floats
        Error on each magnitudes. None if there are no errors.
    Returns
    -------
    fluxes: float or array-like of floats
        The fluxes in Jy.
    errors: float or array-like of floats
        The errors on fluxes in Jy or None.
    """
    magnitudes = np.array(magnitudes)
    fluxes = 10 ** ((8.9 - magnitudes)/2.5)

    if errors_on_magnitudes is not None:
        errors_on_magnitudes = np.array(errors_on_magnitudes)
        errors = np.log(10)/2.5 * fluxes * errors_on_magnitudes
    else:
        errors = None

    return fluxes, errors

In [ ]:
def mag_diff(x,xerr,y,yerr, labels,savefig=True):
    print("Comparing {} band AB 2 arc sec aperture magnitudes from {} and {}".format(labels[2],labels[0],labels[1]))
    fig = plt.figure(figsize=(13, 6))

    gs1 = gridspec.GridSpec(1, 1)
    gs1.update(left=0.05, right=0.4, wspace=0.05)
    ax1 = plt.subplot(gs1[:, :])

    gs2 = gridspec.GridSpec(1, 3)
    gs2.update(left=0.47, right=0.98, hspace=0.05, wspace=0.05)
    ax2 = plt.subplot(gs2[:, :-1])
    #ax3 = plt.subplot(gs2[:, -1], sharey=ax2)
    
    #notInf = ~np.isinf(x) & ~np.isinf(y)
    #x = x[notInf]
    #y = y[notInf]

 
    m = (
        mask 
        #& (merge['help_m_ap_vista_y'] < 30)
        #& (merge['vhs_stellarity']>0.9)
    )

    mean_offset = np.nanmean(x - y)
    median_offset = np.nanmedian(x - y)
    print("Mean offset = {}".format(round(mean_offset,4)))
    print("Median offset = {}".format(round(median_offset,4)))
    
    total_var = xerr**2 + yerr**2 +0.03**2 #the extra 0.03 is an estimate of the abs. cal. error for vista
    unsaturated = (x>18.5) & (x<22.5)

    weighted_bias = np.nansum(
        (
            x[unsaturated]
            -y[unsaturated]
        )/total_var[unsaturated]
    )/np.nansum(1./total_var[unsaturated])
    print('Weighted bias (14.5<mag<15.5) = {}'.format(round(weighted_bias,4)))
    
    
    ax1.plot([0,26],[mean_offset,mean_offset], linestyle='dotted', c='r', alpha=0.5,
             label = 'mean offset = {}'.format(round(mean_offset,4)))
    ax1.plot([0,26],[median_offset,median_offset], linestyle='dashed', c='r', alpha=0.5,
         label = 'median offset = {}'.format(round(median_offset,4)))
    ax1.plot([0,26],[weighted_bias,weighted_bias], linestyle='dashdot', c='r', alpha=0.5,
         label = 'Weighted bias = {}'.format(round(weighted_bias,4)))
    ax1.plot([0,26],[0,0], c='r', label='zero')
    ax1.legend( prop={'size': 12}, loc='upper left')
    ax1.scatter(x,   x - y, 
            alpha=1.0, s=0.3)
    
    
    ax1.set_xlabel('{} [mag]'.format(labels[0]))
    ax1.plot([0,26],[0,0], c='r')
    
    upper = np.min([25.,np.nanmax(x),np.nanmax(y)+1] )
    ax1.set_xlim([np.nanmin(x),np.nanmax(x)])
    ax1.set_ylim([-1,1])
    ax1.set_ylabel('{} - {} [mag]'.format(labels[0],labels[1]))
    
    ax2.hist(x - y,range=[-1,1], bins=60)
    #ax2.set_xlim([-2,2])
    ax2.set_xlabel('{} - {} [mag]'.format(labels[0],labels[1]))
    ax2.axvline(0, c='r', alpha=0.5)
    ax2.axvline(mean_offset,linestyle='dotted', c='r', alpha=0.5)
    ax2.axvline(median_offset,linestyle='dashed', c='r', alpha=0.5)
    ax2.axvline(weighted_bias,linestyle='dashdot', c='r', alpha=0.5)
    
    
    #plt.rc('font', family='serif', serif='Times')
    plt.rc('text') #, usetex=True)
    plt.rc('xtick', labelsize=14)
    plt.rc('ytick', labelsize=14)
    plt.rc('axes', labelsize=14)
    


    if savefig:
        plt.savefig('./figs/mag_compare_{}_{}.pdf'.format(
            labels[0].replace(' ','_'),labels[1].replace(' ','_')), bbox_inches='tight')
        plt.savefig('./figs/mag_compare_{}_{}.png'.format(
            labels[0].replace(' ','_'),labels[1].replace(' ','_')), bbox_inches='tight')
    plt.show()
    
    
    
mask = cat['flag']
mask &= cat['m_ap60_hsc_z'] >18.5
mask &= cat['m_ap60_hsc_z'] <22.5
mask &= cat['m_ap60_vista_z'] >10
mask &= cat['m_ap60_vista_z'] <30
mag_diff(

    cat[mask]['m_ap60_hsc_z'],
    cat[mask]['merr_ap60_hsc_z'],
    cat[mask]['m_ap60_vista_z'],
    cat[mask]['merr_ap60_vista_z'],
    ['HSC-Z 6pix radius ap','VISTA-Z 6pix radius ap', 'Z'])

In [ ]:
mask = cat['flag']
mask &= cat['m_ap90_hsc_z'] >18.5
mask &= cat['m_ap90_hsc_z'] <22.5
mask &= cat['m_ap90_vista_z'] >10
mask &= cat['m_ap90_vista_z'] <30
mag_diff(

    cat[mask]['m_ap90_hsc_z'],
    cat[mask]['merr_ap90_hsc_z'],
    cat[mask]['m_ap90_vista_z'],
    cat[mask]['merr_ap90_vista_z'],
    ['HSC-Z 9pix radius ap','VISTA-Z 9pix radius ap', 'Z'])

In [ ]:
mask = cat['flag']
mask &= cat['m_ap60_hsc_y'] >18.5
mask &= cat['m_ap60_hsc_y'] <22.5
mask &= cat['m_ap60_vista_y'] >10
mask &= cat['m_ap60_vista_y'] <30
mag_diff(

    cat[mask]['m_ap60_hsc_y'],
    cat[mask]['merr_ap60_hsc_y'],
    cat[mask]['m_ap60_vista_y'],
    cat[mask]['merr_ap60_vista_y'],
    ['HSC-Y 6pix radius ap','VISTA-Y 6pix radius ap', 'Y'])

In [ ]:
mask = cat['flag']
mask &= cat['m_ap90_hsc_y'] >18.5
mask &= cat['m_ap90_hsc_y'] <22.5
mask &= cat['m_ap90_vista_y'] >10
mask &= cat['m_ap90_vista_y'] <30
mag_diff(

    cat[mask]['m_ap90_hsc_y'],
    cat[mask]['merr_ap90_hsc_y'],
    cat[mask]['m_ap90_vista_y'],
    cat[mask]['merr_ap90_vista_y'],
    ['HSC-Y 9pix radius ap','VISTA-Y 9pix radius ap', 'Y'])